In [29]:
import numpy as np
import Utilityfunctions as utils
import jax.numpy as jnp
import ssr_kronvec_jax as ssr
import ssr_likelihood_jax as jax_lik
import vanilla as mhn
import scipy.optimize as opt
import regularized_optimization as reg_opt
from jax import lax

In [32]:
n = 3
theta = utils.random_theta(n, 0.0)
n_dat = 100
dat = np.random.binomial(n=1, p=0.6, size=n_dat*(2*n+1))
dat = jnp.array(dat).reshape((n_dat, -1))
dat = dat.at[:,-1].set(1)
dat = jnp.column_stack((dat, jnp.ones(n_dat, dtype=int)))
lam1 = 1
lam2 = 1
indep = np.eye(n+1)
dat

RuntimeError: Unknown backend: 'gpu' requested, but no platforms that are instances of gpu are present. Platforms present are: interpreter,cpu

In [5]:
reg_opt.log_prob_coupled(theta.flatten(), dat, lam1, lam2, 1/n_dat)

10.93000356387289

In [7]:
reg_opt.log_prob_single(theta.flatten(), dat, lam1, 1/n_dat)

7.137805144301969

In [12]:
reg_opt.grad_single(indep.flatten(), dat, lam1, 1/n_dat)

array([-0.43233454, -0.2602528 , -0.43233451, -0.2602528 , -0.2602528 ,
       -0.43233454, -0.43233448, -0.2602528 ,  1.1115787 ,  1.1115787 ,
        1.67924416,  1.1115787 , -0.2602528 , -0.2602528 , -0.43233454,
       -0.43233454])

In [16]:
reg_opt.grad_coupled(indep.flatten(), dat, lam1, lam2, 1/n_dat)

array([-0.73117876, -0.2650083 , -0.12520823, -0.20061699,  1.15698612,
        1.6136843 ,  1.26166034,  1.8741852 ,  0.5452804 ,  0.42692876,
        0.93930984, -0.38747513, -0.30387503, -0.        , -0.        ,
       -0.64272362])

In [25]:
x = opt.minimize(reg_opt.log_prob_coupled, x0 = indep.flatten(), args = (dat, lam1, lam2, 0.0), 
                method = "L-BFGS-B", jac = reg_opt.grad_coupled, options={"maxiter":100, "disp":True})

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           16     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  5.77109D+00    |proj g|=  5.72380D-01


 This problem is unconstrained.



At iterate    1    f=  4.79198D+00    |proj g|=  3.38989D-01

At iterate    2    f=  4.28752D+00    |proj g|=  3.43350D-01

At iterate    3    f=  4.21354D+00    |proj g|=  9.60998D-02

At iterate    4    f=  4.21304D+00    |proj g|=  9.44135D-02

At iterate    5    f=  4.20621D+00    |proj g|=  8.22661D-02

At iterate    6    f=  4.19852D+00    |proj g|=  6.59161D-02

At iterate    7    f=  4.19667D+00    |proj g|=  5.61267D-02

At iterate    8    f=  4.19585D+00    |proj g|=  4.79756D-02

At iterate    9    f=  4.19472D+00    |proj g|=  4.04960D-02

At iterate   10    f=  4.19260D+00    |proj g|=  3.22166D-02

At iterate   11    f=  4.18934D+00    |proj g|=  2.56019D-02

At iterate   12    f=  4.18757D+00    |proj g|=  2.68060D-02

At iterate   13    f=  4.18111D+00    |proj g|=  3.01651D-02

At iterate   14    f=  4.18105D+00    |proj g|=  3.10246D-02

At iterate   15    f=  4.17963D+00    |proj g|=  3.26050D-02

At iterate   16    f=  4.16112D+00    |proj g|=  2.50918D-02

At iter


   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.


  F =   4.1611193847656249     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             


In [ ]:
jnp.around(jnp.reshape(x.x, (n+1, n+1)),2)
x.message

In [ ]:
i = 0
log_theta = jnp.array(theta)
prim_first = bool(dat.at[i, -1].get())
state = dat.at[i, 0:2*n+1].get()
if prim_first:
    latent_state_1 = jnp.append(state.at[1:2*n+1:2].get(), state.at[-1].get())
else:    
    latent_state_1 = state.at[i, 0:2*n+1:2].get()

# Calculate Joint dist at t1
p0 = jnp.zeros(2**int(state.sum()))
p0 = p0.at[0].set(1.0)
pTh1 = lam1 * jax_lik.R_i_inv_vec(log_theta, p0, lam1,  state, transpose = False)

# Get indices of latent_states at t1/t2
latent_dist_1 = ssr.obs_inds_2(pTh1, state, prim_first)
latent_dist_2 = ssr.obs_inds_2(pTh1, state, not prim_first)
latent_dist_2 = latent_dist_2.at[0:latent_dist_2.shape[0]//2].set(0)
latent_size = int(latent_dist_1.sum())

# Get observed dist at t1
pTh1_obs = pTh1.at[jnp.where(latent_dist_1==1, size=latent_size)[0]].get()
nk = pTh1_obs.sum()
pTh1_obs /= nk
pTh2 = mhn.R_inv_vec(log_theta, pTh1_obs, lam2, latent_state_1, False)
q = jnp.zeros_like(pTh2)
q = q.at[-1].set(1/pTh2.at[-1].get())
mhn.R_inv_vec(log_theta, q, lam2, latent_state_1, True)
marg = utils.ssr_marginalize(pTh1, n, state, True, False)
iwas = jnp.zeros_like(marg)
iwas = iwas.at[-1].set(1/marg[-1])
#print(marg)
print(utils.ssr_marginalize_mat_vec_transp(iwas, n, state, True, False))
#print(utils.ssr_obs_dist(pTh1, state, n, True))
obs = utils.ssr_obs_dist_mat_vec(pTh1, state, n, True)
#print(obs)
utils.ssr_obs_dist_mat_vec_transp(pTh1_obs, state, n, True)
jnp.where(latent_dist_1 ==1, pTh1/nk, 0)

In [ ]:
np.arange(0,8).reshape((-1,1), order="F")